In [2]:
import tensorflow as tf
print(tf.__version__)

2.3.0


### GradientTape

y = 2*x*x^T


In [4]:
# 设置变量
x = tf.reshape(tf.Variable(range(4),dtype=tf.float32),(4,1))
print(x)
print(x.shape)

tf.Tensor(
[[0.]
 [1.]
 [2.]
 [3.]], shape=(4, 1), dtype=float32)
(4, 1)


In [5]:

# GradientTape 就是开启自动微分机制
with tf.GradientTape() as t:
    t.watch(x) # 需要求解的变量
    y = 2 * tf.matmul(tf.transpose(x),x)

dy_dx = t.gradient(y,x)
print(dy_dx)

tf.Tensor(
[[ 0.]
 [ 4.]
 [ 8.]
 [12.]], shape=(4, 1), dtype=float32)


In [7]:
# GradientTape 就是开启自动微分机制
# 实现预测的模型
with tf.GradientTape(persistent=True) as g:
    g.watch(x) # 需要求解的变量
    y = x*x
    z =y*y
    dz_dx= g.gradient(z,x)
    dy_dx = g.gradient(y,x)

print(dz_dx,dy_dx)

tf.Tensor(
[[  0.]
 [  4.]
 [ 32.]
 [108.]], shape=(4, 1), dtype=float32) tf.Tensor(
[[0.]
 [2.]
 [4.]
 [6.]], shape=(4, 1), dtype=float32)


In [9]:
# f(x) = a*x**2 + b*x + c的导数

x = tf.Variable(0.0,name="x",dtype=tf.float32)
a = tf.constant(1.0)
b = tf.constant(-2.0)
c = tf.constant(1.0)

with tf.GradientTape() as gt:
    y = a*tf.pow(x,2.0)+b*x+c

dy_dx = gt.gradient(y,x)
print(dy_dx)



tf.Tensor(-2.0, shape=(), dtype=float32)


In [12]:
# f(x) = a*x**2 + b*x + c的导数

x = tf.Variable(0.0,name="x",dtype=tf.float32)
a = tf.constant(1.0)
b = tf.constant(-2.0)
c = tf.constant(1.0)

with tf.GradientTape() as gt:
    # 这里控制相关的输入变量
    gt.watch([a,b,c])
    y = a*tf.pow(x,2.0)+b*x+c

dy_dx,dy_da,dy_db,dy_dc = gt.gradient(y,[x,a,b,c])
dy_dx,dy_da,dy_db,dy_dc

(<tf.Tensor: shape=(), dtype=float32, numpy=-2.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=1.0>)

In [11]:
#二阶求导
x = tf.Variable(0.0,name="x",dtype=tf.float32)
a = tf.constant(1.0)
b = tf.constant(-2.0)
c = tf.constant(1.0)

with tf.GradientTape() as gt_sec:
    with tf.GradientTape() as gt_fir:
        y = a*tf.pow(x,2.0)+b*x+c
    dy_dx = gt_fir.gradient(y,x)
dy2_dx2 = gt_sec.gradient(dy_dx,x)

dy_dx,dy2_dx2


(<tf.Tensor: shape=(), dtype=float32, numpy=-2.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=2.0>)

In [13]:
# 使用autograph 来进行自动微分
@tf.function
def autoGradientFunc(x):
    a = tf.constant(1.0)
    b = tf.constant(-2.0)
    c = tf.constant(1.0)   

    x = tf.cast(x,tf.float32)
    with tf.GradientTape() as gt:
        gt.watch(x)
        y = a*tf.pow(x,2.0)+b*x+c
    dy_dx = gt.gradient(x,y)

    return x,dy_dx


print(autoGradientFunc(x=2))
print(autoGradientFunc(x=-2))



(<tf.Tensor: shape=(), dtype=float32, numpy=2.0>, None)
(<tf.Tensor: shape=(), dtype=float32, numpy=-2.0>, None)


### 梯度下降来寻找最小值

In [4]:
# 求f(x) = a*x**2 + b*x + c的最小值
# 使用梯度下降来寻找最小值

x = tf.Variable(0.0,name="x",dtype=tf.float32)
a = tf.constant(1.0)
b = tf.constant(-2.0)
c = tf.constant(1.0)

# 迭代100次来寻找最小值
# 更新x = x0-dy_dx * learning_rate
learning_rate = tf.constant(0.01)
for _ in range(100):
    #第一步计算梯度需要更新的值的梯度值
    with tf.GradientTape() as gt:
        y = a*tf.pow(x,2.0)+b*x+c
    dy_dx = gt.gradient(y,x)

    # 第二步 更新x = x0-dy_dx * learning_rate
    # assign 是tf中给变量赋值的一种方法
    x.assign(x - dy_dx * learning_rate)
    # tf.print(dy_dx)
tf.print("x=",x)
tf.print("y=",y)
    

x= 0.867380381
y= 0.0183131695


In [ ]:
# 求f(x) = a*x**2 + b*x + c的最小值
# 使用optimizer.apply_gradients

x = tf.Variable(0.0,name="x",dtype=tf.float32)
a = tf.constant(1.0)
b = tf.constant(-2.0)
c = tf.constant(1.0)

# 迭代1000次来寻找最小值
# 更新x = x0-dy_dx * learning_rate

learning_rate = tf.constant(0.01)
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
for _ in range(100):
    with tf.GradientTape() as gt:
        y = a*tf.pow(x,2.0)+b*x+c
    dy_dx = gt.gradient(y,x)
    # 使用optimizer.apply_gradients
    # 更新x = x0-dy_dx * learning_rate
    optimizer.apply_gradients(grads_and_vars=[(dy_dx,x)])
    # x.assign(x - dy_dx * learning_rate)
    tf.print(dy_dx)
    tf.print("x=",x)
    tf.print("y=",y)

In [28]:
# 调用optimizer.minimize优化器了求解
# 这里就将自动微分和反向传播给自动更新了

x = tf.Variable(0.0,name="x",dtype=tf.float32)
#注意f()无参数
def f():   
    a = tf.constant(1.0)
    b = tf.constant(-2.0)
    c = tf.constant(1.0)
    y = a*tf.pow(x,2)+b*x+c
    return(y)

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
for _ in range(100):
    optimizer.minimize(f,[x])
tf.print("y=",f(),"  x=",x)


y= 0.0175879598   x= 0.867380381


In [7]:
# 在autograph中完成最小值求解
x = tf.Variable(0.0,name="x",dtype=tf.float32)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

@tf.function
def minimizef():
    a = tf.constant(1.0)
    b = tf.constant(-2.0)
    c = tf.constant(1.0)

    #注意autograph时使用tf.range(1000)而不是range(1000)
    for _ in tf.range(100):
        with tf.GradientTape() as gt:
            y = a*tf.pow(x,2)+b*x+c
        dy_dx = gt.gradient(y,x)
        optimizer.apply_gradients(grads_and_vars=[(dy_dx,x)])

    # 如果去掉这个就会出现问题，这里的y必须定义
    y = a*tf.pow(x,2)+b*x+c
    return y

tf.print(minimizef())
tf.print(x)  


0.0175879
0.867380381


In [8]:
# 在autograph中完成最小值求解
# 使用optimizer.minimize

x = tf.Variable(0.0,name = "x",dtype = tf.float32)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)   

@tf.function
def f():   
    a = tf.constant(1.0)
    b = tf.constant(-2.0)
    c = tf.constant(1.0)
    y = a*tf.pow(x,2)+b*x+c
    return(y)

# 这里就是进行训练
@tf.function
def train(epoch):  
    for _ in tf.range(epoch):  
        optimizer.minimize(f,[x])
    return(f())


tf.print(train(1000))
tf.print(x)


0
0.999998569
